# 일괄실행용 코드

## 개발자용은 다른 링크를 참고해주세요.

---

# 세종대학교 인공지능 텀프로젝트 - 휴학과 자퇴사이
## Sejong Univ. AI Term Proejct - SejongBHAJ
## Situation Classifier


### Object Detection with Tensorflow API (ODTA)

---


In [1]:
from google.colab import drive
import os

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')
  print('mounted')
else:
  print('already mounted')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
mounted


In [0]:
repo_url = 'https://github.com/Tony607/object_detection_demo'

num_steps = 10000
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
        'batch_size': 12
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
        'batch_size': 12
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
        'batch_size': 8
    },
    
    'faster_rcnn_inception_resnet_v2_atrous': {
        'model_name': 'faster_rcnn_inception_resnet_v2_atrous_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_resnet_v2_atrous_coco.config',
        'batch_size': 8
    },
    
    'ssd_resnet50_v1': {
        'model_name': 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03',
        'pipeline_file': 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync.config',
        'batch_size': 1
    }
}



selected_model = 'ssd_mobilenet_v2'
    
MODEL = MODELS_CONFIG[selected_model]['model_name']
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']    
batch_size = MODELS_CONFIG[selected_model]['batch_size']    

In [3]:
import os
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

/content
Selecting previously unselected package python-bs4.
(Reading database ... 130942 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpa

In [4]:
%cd /content
repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))
!git clone {repo_url}
%cd {repo_dir_path}
!git pull

!rm data/images/train/*
!rm data/images/test/*

/content
Cloning into 'object_detection_demo'...
remote: Enumerating objects: 124, done.
remote: Total 124 (delta 0), reused 0 (delta 0), pack-reused 124
Receiving objects: 100% (124/124), 11.16 MiB | 12.14 MiB/s, done.
Resolving deltas: 100% (55/55), done.
/content/object_detection_demo
Already up to date.


## dataset download

### structure
```
data.tar(1GB)

ㄴtrain.tar(964MB)

    ㄴSejong.mp4#t=0.tfrecord(1.25MB)
        ㄴSejong.mp4#t=0.jpg
        ㄴSejong.mp4#t=0.xml
        
    ㄴSejong.mp4#t=0.5.tfrecord(1.25MB)
    ...
    
ㄴtest.tar(62.1MB)
   ㄴSejong.mp4#t=445.tfrecord(1.25MB)
   ...
   ㄴSejong.mp4#t=469.tfrecord(1.25MB)
   
ㄴlabel_map.pbtxt(1.15KB)
```

In [6]:
!mkdir -p data/annotations
!mkdir -p data/annotations/train
!mkdir -p data/annotations/test
%cd '/content/object_detection_demo/data/annotations/'

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Ib2USZdv1nJcoGJQXNlAercKyZnnw-ZH' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Ib2USZdv1nJcoGJQXNlAercKyZnnw-ZH" -O data.tar && rm -rf /tmp/cookies.txt

import tarfile

tar=tarfile.open("data.tar")
tar.extract("train.tar")
tar.extract("test.tar")
tar.extract("label_map.pbtxt")
tar.close()

!mv train.tar "/content/object_detection_demo/data/annotations/train/train.tar"
!mv test.tar "/content/object_detection_demo/data/annotations/test/test.tar"

%cd '/content/object_detection_demo/data/annotations/train'
tar=tarfile.open("train.tar")
tar.extractall()
tar.close()
!rm train.tar

%cd '/content/object_detection_demo/data/annotations/test'
tar=tarfile.open("test.tar")
tar.extractall()
tar.close()
!rm test.tar

%cd ../../../

/content/object_detection_demo/data/annotations
--2019-06-23 21:41:22--  https://docs.google.com/uc?export=download&confirm=BIyV&id=1Ib2USZdv1nJcoGJQXNlAercKyZnnw-ZH
Resolving docs.google.com (docs.google.com)... 108.177.119.113, 108.177.119.101, 108.177.119.100, ...
Connecting to docs.google.com (docs.google.com)|108.177.119.113|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-4o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/allrpttqnmrn0psbu97vjrjhr07iabt7/1561320000000/09621136847049687709/*/1Ib2USZdv1nJcoGJQXNlAercKyZnnw-ZH?e=download [following]
--2019-06-23 21:41:22--  https://doc-0s-4o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/allrpttqnmrn0psbu97vjrjhr07iabt7/1561320000000/09621136847049687709/*/1Ib2USZdv1nJcoGJQXNlAercKyZnnw-ZH?e=download
Resolving doc-0s-4o-docs.googleusercontent.com (doc-0s-4o-docs.googleusercontent.com)... 108.177.127.132, 2a00:1450:4013:c07::

In [0]:
single='"{}"' #jpgxml
multi="{}" #tfrecord
#print(single, multi)

bracket_train=''
bracket_test=''

In [0]:
train_record_fname_dir ='/content/object_detection_demo/data/annotations/train/'
path=train_record_fname_dir
file_list_tfrecord = []
for file in os.listdir(path):
  if file.endswith(".tfrecord"):
    file_list_tfrecord.append(path+file)
train_record_fname=file_list_tfrecord
#print(train_record_fname)

bracket_train=multi

test_record_fname_dir ='/content/object_detection_demo/data/annotations/test/'
path=test_record_fname_dir
file_list_tfrecord = []
for file in os.listdir(path):
  if file.endswith(".tfrecord"):
    file_list_tfrecord.append(path+file)
test_record_fname=file_list_tfrecord
#print(test_record_fname)

bracket_test=multi

In [0]:
label_map_pbtxt_fname ='/content/object_detection_demo/data/annotations/label_map.pbtxt'

In [10]:
%cd /content/models/research

import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
DEST_DIR = '/content/models/research/pretrained_model'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

/content/models/research


In [0]:
fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
#fine_tune_checkpoint

In [0]:
import os
pipeline_fname = os.path.join('/content/models/research/object_detection/samples/configs/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [0]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

# 오류를 대비한 파이프라인 리셋버튼(백업)

In [0]:
!cp {pipeline_fname} {pipeline_fname}.bak

In [0]:
#!cp {pipeline_fname}.bak {pipeline_fname}

# !경고!

## 데이터셋 업로드 여러 개의 tfrecord 파일을 업로드하였다면
## 정규표현식 대입 수정이 필요합니다.

### 1개일 경우:
> input_path: "{}" (큰따음표)

### 여러 개일 경우
> input_path: '{}' (작은따음표)

In [15]:
import re

num_classes = get_num_classes(label_map_pbtxt_fname)

#ERROR ! The number of calss of each label map pbtxt are different! You must have the same class to train.

with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(train.record)(.*?")', 'input_path: '+bracket_train.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(val.record)(.*?")', ('input_path: '+bracket_test).format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #s = re.sub('initial_learning_rate: 0.004', 'initial_learning_rate: 0.01', s)
    f.write(s)

W0623 21:42:32.275906 139690344122240 deprecation_wrapper.py:119] From /content/models/research/object_detection/utils/label_map_util.py:132: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [16]:
%cd /
model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

/


In [17]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2019-06-23 21:43:01--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.224.236.142, 54.174.228.92, 34.226.180.131, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.224.236.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17556757 (17M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  16.74M  16.6MB/s    in 1.0s    

2019-06-23 21:43:03 (16.6 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [17556757/17556757]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = model_dir
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [20]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://c519fd8d.ngrok.io


In [47]:
!python /content/models/research/object_detection/model_main.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

W0624 00:12:06.970629 140495266756480 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0624 00:12:06.995630 140495266756480 deprecation_wrapper.py:119] From /content/models/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0624 00:12:07.006050 140495266756480 deprecation_wrapper.py:119] From /content/models/research/slim/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0624 00:12:07.017573 140495266756480 deprecation_wrapper.py:119] From /content/models/research/object_detection/model_main.py:109: The name tf.app.run 

In [48]:
import re
import numpy as np

output_directory = './fine_tuned_model'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
!python /content/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

training/model.ckpt-10000
W0624 02:08:14.458543 140362188785536 deprecation_wrapper.py:119] From /content/models/research/slim/nets/inception_resnet_v2.py:373: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W0624 02:08:14.469620 140362188785536 deprecation_wrapper.py:119] From /content/models/research/slim/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0624 02:08:14.480847 140362188785536 deprecation_wrapper.py:119] From /content/models/research/object_detection/export_inference_graph.py:156: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0624 02:08:14.481548 140362188785536 deprecation_wrapper.py:119] From /content/models/research/object_detection/export_inference_graph.py:139: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0624 02:08:14.501060 140362188785536 deprecation_wrapper.py:119] From /content/models/research/object_de

In [0]:
import os

pb_fname = os.path.join(os.path.abspath(output_directory), "frozen_inference_graph.pb")
assert os.path.isfile(pb_fname), '`{}` not exist'.format(pb_fname)

# 테스트 이미지 업로드

In [50]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = pb_fname

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = label_map_pbtxt_fname

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  os.path.join(repo_dir_path, "test")
#PATH_TO_TEST_IMAGES_DIR = '/content/gdrive/"My Drive"/ai_project/data/'

%cd {PATH_TO_TEST_IMAGES_DIR}
#https://drive.google.com/open?id=1jBujyNnJs6twVLXHFGO7g84iMynFBOF1
!rm -rf *
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jBujyNnJs6twVLXHFGO7g84iMynFBOF1' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1jBujyNnJs6twVLXHFGO7g84iMynFBOF1" -O test_sejong.tar && rm -rf /tmp/cookies.txt
tar=tarfile.open('test_sejong.tar')
tar.extractall()
!rm test_sejong.tar

assert os.path.isfile(pb_fname)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)

/content/object_detection_demo/test
--2019-06-24 02:08:33--  https://docs.google.com/uc?export=download&confirm=LNgQ&id=1jBujyNnJs6twVLXHFGO7g84iMynFBOF1
Resolving docs.google.com (docs.google.com)... 108.177.119.139, 108.177.119.101, 108.177.119.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.119.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-4o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hb3nmbbe39jk9vkfscfcik2j2uideofn/1561341600000/09621136847049687709/*/1jBujyNnJs6twVLXHFGO7g84iMynFBOF1?e=download [following]
--2019-06-24 02:08:33--  https://doc-14-4o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/hb3nmbbe39jk9vkfscfcik2j2uideofn/1561341600000/09621136847049687709/*/1jBujyNnJs6twVLXHFGO7g84iMynFBOF1?e=download
Resolving doc-14-4o-docs.googleusercontent.com (doc-14-4o-docs.googleusercontent.com)... 108.177.127.132, 2a00:1450:4013:c07::84
Connectin

In [0]:
#print(TEST_IMAGE_PATHS)

In [0]:
TEST_IMAGE_PATHS.sort()

In [52]:
%cd /content/models/research/object_detection

import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops


# This is needed to display the images.
%matplotlib inline


from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util


detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')


label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(
    label_map, max_num_classes=num_classes, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    #(im_width, im_height, _) = image.shape
    return np.array(image).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

# Size, in inches, of the output images.
#IMAGE_SIZE = (12, 8)

def run_inference_for_single_image(image, graph):
    with graph.as_default():
        with tf.Session() as sess:
            # Get handles to input and output tensors
            ops = tf.get_default_graph().get_operations()
            all_tensor_names = {
                output.name for op in ops for output in op.outputs}
            tensor_dict = {}
            for key in [
                'num_detections', 'detection_boxes', 'detection_scores',
                'detection_classes', 'detection_masks'
            ]:
                tensor_name = key + ':0'
                if tensor_name in all_tensor_names:
                    tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
                        tensor_name)
            if 'detection_masks' in tensor_dict:
                # The following processing is only for single image
                detection_boxes = tf.squeeze(
                    tensor_dict['detection_boxes'], [0])
                detection_masks = tf.squeeze(
                    tensor_dict['detection_masks'], [0])
                # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
                real_num_detection = tf.cast(
                    tensor_dict['num_detections'][0], tf.int32)
                detection_boxes = tf.slice(detection_boxes, [0, 0], [
                                           real_num_detection, -1])
                detection_masks = tf.slice(detection_masks, [0, 0, 0], [
                                           real_num_detection, -1, -1])
                detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                    detection_masks, detection_boxes, image.shape[0], image.shape[1])
                detection_masks_reframed = tf.cast(
                    tf.greater(detection_masks_reframed, 0.5), tf.uint8)
                # Follow the convention by adding back the batch dimension
                tensor_dict['detection_masks'] = tf.expand_dims(
                    detection_masks_reframed, 0)
            image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

            # Run inference
            output_dict = sess.run(tensor_dict,
                                   feed_dict={image_tensor: np.expand_dims(image, 0)})

            # all outputs are float32 numpy arrays, so convert types as appropriate
            output_dict['num_detections'] = int(
                output_dict['num_detections'][0])
            output_dict['detection_classes'] = output_dict[
                'detection_classes'][0].astype(np.uint8)
            output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
            output_dict['detection_scores'] = output_dict['detection_scores'][0]
            if 'detection_masks' in output_dict:
                output_dict['detection_masks'] = output_dict['detection_masks'][0]
    return output_dict

/content/models/research/object_detection


In [30]:
!cd /; cd "content/gdrive/My Drive/Colab Notebooks/"; ls

'[5-2]<선형분류>Linear Classification'
'[5]<선형분류>[과제]과제2_Skeleton'
'[6-2]<NN>tensorflow_Learning_Tips'
 checkpoint
'인공지능 CNN 퀴즈 문제_17011832_김남훈.ipynb'
'인공지능 CNN 퀴즈2 문제1_17011832_김남훈'
'인공지능 CNN 퀴즈2 문제2_17011832_김남훈'
 data-zoo-v2.csv
 diabetes.csv
 object_detection_TensorflowAPI
 Object_detection_with_Tensorflow_API_at_colab_SejongBHAJ_Backup
 ODTA_SejongBHAJ.ipynb
'ODTA_SejongBHAJ_OneClick(for Marking)'
 price_data_yk.csv
 saved.cpkt.data-00000-of-00001
 saved.cpkt.index
 saved.cpkt.meta
 SSD-Tensorflow
'tensorflow-object-detection-training-colab.ipynb 개정판'
 Untitled0.ipynb
 Untitled10.ipynb
 Untitled11.ipynb
 Untitled12.ipynb
 Untitled13.ipynb
 Untitled14.ipynb
 Untitled15.ipynb
 Untitled16.ipynb
 Untitled1.ipynb
 Untitled2.ipynb
 Untitled3.ipynb
 Untitled4.ipynb
 Untitled5.ipynb
 Untitled6.ipynb
 Untitled7.ipynb
 Untitled8.ipynb
 Untitled9.ipynb
'(Unused)Object_detction_TensorflowAPI_train.ipynb'


In [53]:
!mkdir "/content/gdrive/My Drive/samplefolder"

mkdir: cannot create directory ‘/content/gdrive/My Drive/samplefolder’: File exists


In [0]:
#@title
%matplotlib inline

%cd "/content/gdrive/My Drive/samplefolder"

import numpy as np
import codecs
import json
from collections import OrderedDict
import cv2
print(cv2.__version__)


i=0
#Maximum: 20pics
for image_path in TEST_IMAGE_PATHS:
  with Image.open(image_path) as image:

    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = load_image_into_numpy_array(image)
    #image_np=image
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    
    '''
    #----------------------classification_to_toss_depth-------------
    
    # Actual detection.
    output_dict = run_inference_for_single_image(image_np, detection_graph)

    threshold=0.5
    
    f=OrderedDict()
    f["num_detections"]=output_dict["num_detections"]      
    
    boxes=[]
    scores=[]
    classes=[]
    
    for i in range(output_dict["num_detections"]):  
      det_box=output_dict["detection_boxes"][i]
      det_score=output_dict["detection_scores"][i]
      det_class=output_dict["detection_classes"][i]
    
      #threshold check to output filter
      
      if det_score<0.5:
        continue
      
      boxes.append(det_box)
      scores.append(det_score)
      classes.append(det_class)
      
    f["detection_boxes"]=boxes
    f["detection_scores"]=scores
    f["detection_classes"]=classes

    json.dumps(f, ensure_ascii=False, indent="\t")
    
    #--------------------------------------------
    '''
    
    # 감지 결과 json으로 저장
    f=open(os.path.basename(image_path).split('.')[0]+"_test.txt", 'w')
    f.write(str(output_dict["num_detections"]))
    f.write("\n")
    f.write(str(output_dict["detection_boxes"]))
    f.write("\n")
    f.write(str(output_dict["detection_scores"]))
    f.write("\n")
    f.write(str(output_dict["detection_classes"]))
    f.write("\n")
    f.close()
    
        
    '''
    #--------------dangerous classification with thickness and color------
    flag=False
    dangerous_classnum=[14, 18, 26] #person, bycicle, car
    
    if dangerous_classnum in output_dict['detection_classes']:
      flag=True
      line_thickness=10
      groundtruth_box_visualization_color="red"
      
    #-------------------------------------------------  
    '''

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=5,
    )
    plt.imsave(os.path.basename(image_path).split('.')[0]+'_test.jpg', image_np)
    print(str(round((i/2402*100), 2))+"%")
    i=i+1
    #plt.imshow(image_np); plt.show(); plt.close()

/content/gdrive/My Drive/samplefolder
3.4.3
0.0%
0.04%
0.08%
0.12%
0.17%
0.21%
0.25%
0.29%
0.33%
0.37%
0.42%
0.46%
0.5%
0.54%
0.58%
0.62%
0.67%
0.71%
0.75%
0.79%
0.83%
0.87%
0.92%
0.96%
1.0%
1.04%
1.08%
1.12%
1.17%
1.21%
1.25%
1.29%
1.33%
1.37%
1.42%
1.46%
1.5%
1.54%
1.58%
1.62%
1.67%
1.71%
1.75%
1.79%
1.83%
1.87%
1.92%
1.96%
2.0%
2.04%
2.08%
2.12%
2.16%
2.21%
2.25%
2.29%
2.33%
2.37%
2.41%
2.46%
2.5%
2.54%
2.58%
2.62%
2.66%
2.71%
2.75%
2.79%
2.83%
2.87%
2.91%
2.96%
3.0%
3.04%
3.08%
3.12%
3.16%
3.21%
3.25%
3.29%
3.33%
3.37%
3.41%
3.46%
3.5%
3.54%
3.58%
3.62%
3.66%
3.71%
3.75%
3.79%
3.83%
3.87%
3.91%
3.96%
4.0%
4.04%
4.08%
4.12%
4.16%
4.2%
4.25%
4.29%
4.33%
4.37%
4.41%
4.45%
4.5%
4.54%
4.58%
4.62%
4.66%
4.7%
4.75%
4.79%
4.83%
4.87%
4.91%
4.95%
5.0%
5.04%
5.08%
5.12%
5.16%
5.2%
5.25%
5.29%
5.33%
5.37%
5.41%
5.45%
5.5%
5.54%
5.58%
5.62%
5.66%
5.7%
5.75%
5.79%
5.83%
5.87%
5.91%
5.95%
6.0%
6.04%
6.08%
6.12%
6.16%
6.2%
6.24%
6.29%
6.33%
6.37%
6.41%
6.45%
6.49%
6.54%
6.58%
6.62%
6.66%
6.7%
6.7

In [45]:
!

frame00000.json      frame00801_test.jpg  frame01603.json
frame00000_test.jpg  frame00802.json	  frame01603_test.jpg
frame00001.json      frame00802_test.jpg  frame01604.json
frame00001_test.jpg  frame00803.json	  frame01604_test.jpg
frame00002.json      frame00803_test.jpg  frame01605.json
frame00002_test.jpg  frame00804.json	  frame01605_test.jpg
frame00003.json      frame00804_test.jpg  frame01606.json
frame00003_test.jpg  frame00805.json	  frame01606_test.jpg
frame00004.json      frame00805_test.jpg  frame01607.json
frame00004_test.jpg  frame00806.json	  frame01607_test.jpg
frame00005.json      frame00806_test.jpg  frame01608.json
frame00005_test.jpg  frame00807.json	  frame01608_test.jpg
frame00006.json      frame00807_test.jpg  frame01609.json
frame00006_test.jpg  frame00808.json	  frame01609_test.jpg
frame00007.json      frame00808_test.jpg  frame01610.json
frame00007_test.jpg  frame00809.json	  frame01610_test.jpg
frame00008.json      frame00809_test.jpg  frame01611.json
frame0